### simple example
from https://medium.datadriveninvestor.com/a-data-scientists-approach-for-algorithmic-trading-using-deep-reinforcement-learning-an-be8da40b2230


In [2]:
#imports
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

import json

In [21]:
#!cat key_dir+key_file
import os

tmp = '/Users/piers/Google Drive/My Drive/'
#tmp = '/Users/piers/Library/CloudStorage/GoogleDrive-piers.stobbs@gmail.com/My Drive'
os.listdir(tmp)
#arr = os.listdir('/Users/piers/\"Google Drive\"/\"My Drive\"/personal/code/')
#arr

['personal', '.DS_Store', 'public', 'work', 'travel']

In [37]:
#tickers and indicators
from finrl import config
from finrl import config_tickers
print(dir(config_tickers))
ticker_list = config_tickers.DOW_30_TICKER
print(ticker_list)
TECHNICAL_INDICATORS_LIST = config.INDICATORS
print(TECHNICAL_INDICATORS_LIST)

['CAC_40_TICKER', 'CSI_300_TICKER', 'DAX_30_TICKER', 'DOW_30_TICKER', 'FX_TICKER', 'HSI_50_TICKER', 'LQ45_TICKER', 'MDAX_50_TICKER', 'NAS_100_TICKER', 'SDAX_50_TICKER', 'SINGLE_TICKER', 'SP_500_TICKER', 'SRI_KEHATI_TICKER', 'SSE_50_TICKER', 'TECDAX_TICKER', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'annotations']
['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']
['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


In [25]:
#load key
key_dir = '/Users/piers/Google Drive/My Drive/personal/code/'
key_file = 'alpaca.json'
f = open(key_dir+key_file)
key = json.load(f)  
# returns JSON object as 
# a dictionary

In [26]:
# pick data source
API_KEY=key['api_key']
API_SECRET=key['api_secret']
APCA_API_BASE_URL='https://paper-api.alpaca.markets'

DP = DataProcessor(data_source = 'alpaca',
                   API_KEY = API_KEY,
                   API_SECRET = API_SECRET,
                   APCA_API_BASE_URL = APCA_API_BASE_URL)

Alpaca successfully connected


In [30]:
#get data
data = DP.download_data(start_date = '2021-10-01',
                        end_date = '2021-10-05',
                        ticker_list = ticker_list,
                        time_interval= '1Min')

/Users/piers/gitprojects/explore/explore_deeprl/FinRL-master/finrl/meta/data_processors/processor_alpaca.py:51: FutureWarning: Timedelta.delta is deprecated and will be removed in a future version.
  if pd.Timedelta(time_interval).delta < day_delta:


In [32]:
data.head()

,timestamp,open,high,low,close,volume,trade_count,vwap,tic
0,2021-10-01 09:30:00-04:00,141.83,142.040,141.51,141.89,1271789,7488,141.807895,AAPL
1,2021-10-01 09:31:00-04:00,141.89,141.960,141.41,141.43,523240,4755,141.695800,AAPL
2,2021-10-01 09:32:00-04:00,141.44,141.530,141.00,141.00,697099,7015,141.271372,AAPL
3,2021-10-01 09:33:00-04:00,141.01,141.165,140.75,140.79,796857,9188,140.952407,AAPL
4,2021-10-01 09:34:00-04:00,140.80,140.890,140.58,140.74,586007,4875,140.743007,AAPL


In [ ]:
# cleanup and features
data = DP.clean_data(data)
data = DP.add_technical_indicator(data, TECHNICAL_INDICATORS_LIST)
data = DP.add_vix(data)

In [35]:
data.head()

,timestamp,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,VIXY
0,2021-10-01 09:30:00-04:00,141.83,142.04,141.51,141.89,1271789.0,AAPL,0.0,NaN,NaN,NaN,NaN,NaN,141.89,141.89,22.17
1,2021-10-01 09:30:00-04:00,212.69,214.05,212.68,212.71,41899.0,AMGN,0.0,NaN,NaN,NaN,NaN,NaN,212.71,212.71,22.17
2,2021-10-01 09:30:00-04:00,168.86,168.86,168.86,168.86,0.0,AXP,0.0,NaN,NaN,NaN,NaN,NaN,168.86,168.86,22.17
3,2021-10-01 09:30:00-04:00,222.85,223.92,222.85,223.77,189914.0,BA,0.0,NaN,NaN,NaN,NaN,NaN,223.77,223.77,22.17
4,2021-10-01 09:30:00-04:00,192.90,194.00,192.90,193.73,72151.0,CAT,0.0,NaN,NaN,NaN,NaN,NaN,193.73,193.73,22.17


In [36]:
#transform to numpy
price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix='True')

Successfully transformed into array


In [38]:
#train drl agent
#demo for elegantrl
ERL_PARAMS = {"learning_rate": 3e-5,
              "batch_size": 2048,
              "gamma": 0.99,
              "seed":312,
              "net_dimension":512}
train(start_date = '2021-10-11',
      end_date = '2021-10-15',
      ticker_list = ticker_list,
      data_source = 'alpaca',
      time_interval= '1Min',
      technical_indicator_list= TECHNICAL_INDICATORS_LIST,
      drl_lib='elegantrl',
      env=env,
      model_name='ppo',
      API_KEY = API_KEY,
      API_SECRET = API_SECRET,
      APCA_API_BASE_URL = APCA_API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl',
      total_timesteps=1e5)


NameError: name 'train' is not defined